## Maximum Likelyhood for PGM

In [183]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Class definitions - RV, Distribution and Category

In [196]:
class RV:
    def __init__(self, cattup,name):
        self.name = name
        self.categories = cattup
        self.catcountdict = {}
        for category in self.categories:
            self.catcountdict[category] = Category(category)
    def __str__(self):
        return_string = " \t < RV: " + self.name + " > \n"
        count = 0
        for category in self.catcountdict:
            return_string = return_string + str(category) + "\t\t"+ str(self.catcountdict[category].count) + "\n"
            count = count + self.catcountdict[category].count
        return_string = return_string + "total count: " + str(count)
        return return_string
    def clear_count(self):
        for cat in self.catcountdict:
            self.catcountdict[cat].count = 0
        
class Distribution:
    def __init__(self,randomvar):
        self.name = randomvar.name
        categories = randomvar.categories
        catcountdict = randomvar.catcountdict
        distdict = {}
        total = 0
        for category in categories:
            total = total + catcountdict[category].count
        for category in categories:
            distdict[category] = catcountdict[category].count/(1.0*total)
        
        self.categories = categories
        self.distdict = distdict
        self.total = total
    def __str__(self):
        return_string = " \t < Distribution: " + self.name + " > \n"
        for category in self.distdict:
            return_string = return_string + str(category) + "\t\t"+ str(self.distdict[category]) + "\n"
        return return_string
    def to_csv(self):
        table = []
        for category in self.distdict:
            table.append([category,self.distdict[category]])
        df = pd.DataFrame(data=table, columns=['category','prob'])
        name = "prior_" + self.name + ".csv"
        df.to_csv(name,sep=',')
    def to_emdw_csv(self):
        table = []
        ind = 0
        for category in self.distdict:
            table.append([ind,self.distdict[category]])
            ind = ind +1
        df = pd.DataFrame(data=table)
        name = "prior_" + self.name
        print(df.to_string(index=False))
        df.to_csv(name,sep=' ',index=False,header=False)

class Category:
    def __init__(self,category,catcount=0):
        self.name = category
        self.count = catcount
    def increment(self):
        self.count = self.count + 1
        




### Read in the graph

In [197]:
x = pd.read_csv('car2.csv')
# x = pd.read_csv('g.csv')


### Store the possible categories per RV

In [198]:
# find out how many categories each RV can take
# rvcategorydict = {}
# for rv in x.columns:
#     rvcategorydict[rv] = tuple(set([item[0] for item in x[[rv]].values]))

rvcategorydict = {'safety': ('low', 'med', 'high'), 'lug_boot': ('small', 'med', 'big'), 'persons': ('2', '4','more'), 'verdict': ('unacc', 'acc', 'good', 'vgood'), 'doors': ('2', '4', '5more', '3'), 'buying': ('vhigh', 'high', 'med', 'low'), 'maint': ('vhigh', 'high', 'med', 'low')}

for rv in rvcategorydict:
    print(rv,rvcategorydict[rv])
    
emdwrvcategorydict = {}
for rv, categories in rvcategorydict.items():
    emdwcategorydict = {}
    for ind, category in enumerate(categories):
        emdwcategorydict[category] = ind
    emdwrvcategorydict[rv] = emdwcategorydict
    
# for rv, emdwcategorydict in emdwrvcategorydict.items():
#     for category,ind in emdwcategorydict.items():
#         print(category,ind)

# for child,parents in family.items():
#     print("------->",child)
#     for rv in parents:
#         print("--------------------->",rv)
#         for category in rvcategorydict[rv]:
#             print(category,emdwrvcategorydict[rv][category])
        
        


safety ('low', 'med', 'high')
lug_boot ('small', 'med', 'big')
doors ('2', '4', '5more', '3')
verdict ('unacc', 'acc', 'good', 'vgood')
persons ('2', '4', 'more')
buying ('vhigh', 'high', 'med', 'low')
maint ('vhigh', 'high', 'med', 'low')


### Set up the RV dictionary

In [199]:
rvdictionary = {}

for rv in rvcategorydict:
    rvdictionary[rv] = RV(rvcategorydict[rv],name=rv)
    

## Calculate the prior probabilities

In [200]:
for rv in x.columns:
    for item in x[[rv]].values:
        rvdictionary[rv].catcountdict[item[0]].increment()

distributiondict = {}

for rv in rvdictionary:
    distributiondict[rv] = Distribution(rvdictionary[rv])

# for rv in rvdictionary:
#     print(rvdictionary[rv])
    
for rv in distributiondict:
    print(distributiondict[rv])
    Distribution.to_emdw_csv(distributiondict[rv])

 	 < Distribution: safety > 
med		0.33299284984678246
low		0.33401430030643514
high		0.33299284984678246

0         1
0  0.332993
1  0.334014
2  0.332993
 	 < Distribution: lug_boot > 
med		0.33350357507660877
small		0.33401430030643514
big		0.3324821246169561

0         1
0  0.333504
1  0.334014
2  0.332482
 	 < Distribution: doors > 
2		0.278855975485189
4		0.23442288049029622
5more		0.236976506639428
3		0.24974463738508682

0         1
0  0.278856
1  0.234423
2  0.236977
3  0.249745
 	 < Distribution: verdict > 
good		0.035240040858018386
vgood		0.03319713993871297
acc		0.2160367722165475
unacc		0.7155260469867212

0         1
0  0.035240
1  0.033197
2  0.216037
3  0.715526
 	 < Distribution: persons > 
more		0.3289070480081716
2		0.33963227783452504
4		0.33146067415730335

0         1
0  0.328907
1  0.339632
2  0.331461
 	 < Distribution: buying > 
vhigh		0.27425944841675176
high		0.2844739530132789
low		0.22063329928498468
med		0.22063329928498468

0         1
0  0.274259
1  0.284

## Maximum likelyhood

### Define the graph structure

In [211]:
# family = {'C':['A','B'], 'D':['C'],'F':['D','B','C']}
# family = {'verdict':['buying','maint', 'doors', 'persons', 'lug_boot', 'safety']}
family = {'verdict':['buying', 'doors', 'persons', 'safety'],'buying':['maint','lug_boot']}

In [212]:
for child in family:
    print("child: ",child,"\nparents: ", family[child])

child:  verdict 
parents:  ['buying', 'doors', 'persons', 'safety']
child:  buying 
parents:  ['maint', 'lug_boot']


### Compute the maximum likelyhood counts

In [213]:
class ChildContainer:
    def __init__(self,rvobj, childcategorydict,childrvdict):
        self.rvobj = rvobj
        self.catdict = childcategorydict

In [214]:
# graphdict = {'child': child_categorypartup_dict}
graph_dict = {}

for child, parents in family.items():
    #child_categorypartup_dict = {'child_category (e.g. 'unacc')': categorypartup_list}
    child_categorypartup_dict = {}

    #need to enumerate to get access to the child_category so that the partup can be appended to the correct categorypartup_list
    for (i,parent_observations) in enumerate(x[[*parents]].values):
        child_category = x[[child]].values[i][0]
        if child_category in child_categorypartup_dict:
            child_categorypartup_dict[child_category].append((tuple(parents),tuple(parent_observations)))
        else:
            #categorypartup_list = [categorypartup,categorypartup,...]
            categorypartup_list = [(tuple(parents),tuple(parent_observations))]
            child_categorypartup_dict[child_category] = categorypartup_list

    graph_dict[child] = child_categorypartup_dict

#graph_counts_dict = {'child': counts_dict}
graph_counts_dict = {}
for child, child_categorypartup_dict in graph_dict.items():
    #child_counts_list = [partup_count_dict, child_categorycount_dict]
    child_counts_list = []
    #partup_count_dict = {'partup':count}
    partup_count_dict = {}
    #child_categorycount_dict = {'child_category': categorypartup_count_dict}
    child_categorycount_dict = {}
    for child_category, categorypartup_list in child_categorypartup_dict.items():
        print("---------------->", child_category)
     
        #categorypartup_count_dict = {'partup':count}
        categorypartup_count_dict = {}
        print("--------------------> child_category:", child_category)
        print("length: ",len(categorypartup_list))
        print("length of set:", len(set(categorypartup_list)))
        
        for partup in categorypartup_list:
#             print("countpartup:",partup)
            # add to specific category
            if partup in categorypartup_count_dict:
                categorypartup_count_dict[partup] = categorypartup_count_dict[partup] + 1
            else:
                categorypartup_count_dict[partup] = 1
           
                
            # add to generic list
            if partup in partup_count_dict:
                partup_count_dict[partup] = partup_count_dict[partup] + 1
            else:
                partup_count_dict[partup] = 1
#             print(child_category,partup[1],categorypartup_count_dict[partup],partup_count_dict[partup])
        print("adding cat",child_category)        
        child_categorycount_dict[child_category] = categorypartup_count_dict
        print("length of child_categorycout_dict[child_category]: ",len(child_categorycount_dict[child_category]))
    print("length of partup_count_dict[category]: ",len(partup_count_dict))
    for category in child_categorycount_dict:
        print(category)
    child_counts_list = [partup_count_dict, child_categorycount_dict]
    graph_counts_dict[child] =  child_counts_list
    
    
# mle_graph_dict = {}      
# for child, child_counts_list in graph_counts_dict.items():
#     #child_mle_dict = {'small_family_tup': mle}
#     child_mle_dict = {}
#     partup_count_dict =  child_counts_list[0]
#     child_category_count_dict = child_counts_list[1]
#     for category, category_partup_count_dict in child_category_count_dict.items():
#         for partup, cat_count in category_partup_count_dict.items():
#             small_family_tup = (tuple([child]+[par for par in partup[0]]),tuple([category]+[cat for cat in partup[1]]))
#             child_mle_dict[small_family_tup] = cat_count/partup_count_dict[partup]
# #             print(partup[1])
#             print(category, small_family_tup[1],cat_count,partup_count_dict[partup],",", cat_count/partup_count_dict[partup])
#     mle_graph_dict[child] = child_mle_dict 
            
    


----------------> good
--------------------> child_category: good
length:  69
length of set: 32
adding cat good
length of child_categorycout_dict[child_category]:  32
----------------> vgood
--------------------> child_category: vgood
length:  65
length of set: 16
adding cat vgood
length of child_categorycout_dict[child_category]:  16
----------------> acc
--------------------> child_category: acc
length:  423
length of set: 72
adding cat acc
length of child_categorycout_dict[child_category]:  72
----------------> unacc
--------------------> child_category: unacc
length:  1401
length of set: 130
adding cat unacc
length of child_categorycout_dict[child_category]:  130
length of partup_count_dict[category]:  144
acc
good
vgood
unacc
----------------> vhigh
--------------------> child_category: vhigh
length:  537
length of set: 12
adding cat vhigh
length of child_categorycout_dict[child_category]:  12
----------------> high
--------------------> child_category: high
length:  557
length of

In [215]:
for child, child_counts_list in graph_counts_dict.items():
    for category in child_counts_list[1]:
        print(category)
    
mle_graph_dict = {}      
for child, child_counts_list in graph_counts_dict.items():
    #child_mle_dict = {'small_family_tup': mle}
    child_mle_dict = {}
    partup_count_dict =  child_counts_list[0]
    child_category_count_dict = child_counts_list[1]
    counter = 0
    for category, category_partup_count_dict in child_category_count_dict.items():
       
        for partup, cat_count in category_partup_count_dict.items():
            counter = counter + 1
#             print(category, partup[1],cat_count/partup_count_dict[partup])
            small_family_tup = (tuple([child]+[par for par in partup[0]]),tuple([category]+[cat for cat in partup[1]]))
            child_mle_dict[small_family_tup] = cat_count/partup_count_dict[partup]
#             print(partup[1])
#             print(category, small_family_tup[1],cat_count,partup_count_dict[partup],",", cat_count/partup_count_dict[partup])
        print(category,counter,len(category_partup_count_dict))
    mle_graph_dict[child] = child_mle_dict 
            
    


acc
good
vgood
unacc
vhigh
high
low
med
acc 72 72
good 104 32
vgood 120 16
unacc 250 130
vhigh 12 12
high 24 12
low 36 12
med 48 12


### Print the MLE conditional probabilities for each child

In [218]:
familytable = {}
familycolumns = {}
emdwfamilytable = {}

for child, parents in family.items():
    familycolumns[child] = [child] + parents

for child, child_mle_dict in mle_graph_dict.items():
    table = []
    emdwtable = []
    col = []
    for entry,condprob in child_mle_dict.items():
        row = []
        emdwrow = []
        truecol = []
        for i,val in enumerate(entry[1]):
            row.append(val)
            truecol.append(entry[0][i])
            emdwrow.append(emdwrvcategorydict[entry[0][i]][val])
        row.append(condprob)
        emdwrow.append(condprob)
        table.append(row)
        emdwtable.append(emdwrow)
        col = truecol
    familytable[child] = table
    familycolumns[child] = col
    emdwfamilytable[child] = emdwtable
    
dfdict = {}
for child, emdwtable in emdwfamilytable.items():
    df = pd.DataFrame(data=emdwtable, columns=familycolumns[child] + ['condprob'])
#     df = pd.DataFrame(data=emdwtable)
    name = "emdw_" + child + "_given_"
    for parent in family[child]:
        name = name +"_" + parent
#     name = name + ".txt"
    print(name)
    dfdict[child] = df
    df.to_csv(name,sep=' ',index=False,header=False)
    
dfdict1 = {}
for child, table in familytable.items():
    df = pd.DataFrame(data=table, columns=familycolumns[child] + ['condprob'])
    name = child + "_given_"
    for parent in family[child]:
        name = name +"_" + parent
    name = name + ".csv"
    print(name)
    df.to_csv(name,sep=',')
    dfdict1[child] = df
    

    


emdw_verdict_given__buying_doors_persons_safety
emdw_buying_given__maint_lug_boot
verdict_given__buying_doors_persons_safety.csv
buying_given__maint_lug_boot.csv


In [219]:
for child in dfdict1:
    print(dfdict1[child].to_string(index=False))

verdict buying  doors persons safety  condprob
 vgood    low  5more       4   high  0.500000
   acc  vhigh      3    more    med  0.333333
   acc  vhigh  5more    more   high  0.500000
 unacc  vhigh  5more       2    low  1.000000
 unacc  vhigh      3       2    med  0.894737
   acc  vhigh      2    more   high  0.333333
 unacc    med      3       4    low  1.000000
 unacc    low      2       4    med  0.166667
   acc    low      3       4   high  0.416667
  good    low  5more       4   high  0.166667
 unacc    med      2       4    low  1.000000
 unacc   high      3    more    med  0.600000
 unacc   high  5more    more    med  0.647059
   acc  vhigh      2       2    med  0.210526
 unacc   high  5more       4    low  1.000000
   acc    low      2    more   high  0.250000
   acc  vhigh      4    more   high  0.500000
   acc    med  5more    more    med  0.666667
   acc    low  5more    more    med  0.583333
 vgood    med      2    more   high  0.166667
 unacc    low  5more    more    m

In [220]:
for child in dfdict:
    print(dfdict[child].to_string(index=False))

verdict  buying  doors  persons  safety  condprob
      3       3      2        1       2  0.500000
      1       0      3        2       1  0.333333
      1       0      2        2       2  0.500000
      0       0      2        0       0  1.000000
      0       0      3        0       1  0.894737
      1       0      0        2       2  0.333333
      0       2      3        1       0  1.000000
      0       3      0        1       1  0.166667
      1       3      3        1       2  0.416667
      2       3      2        1       2  0.166667
      0       2      0        1       0  1.000000
      0       1      3        2       1  0.600000
      0       1      2        2       1  0.647059
      1       0      0        0       1  0.210526
      0       1      2        1       0  1.000000
      1       3      0        2       2  0.250000
      1       0      1        2       2  0.500000
      1       2      2        2       1  0.666667
      1       3      2        2       1  0.583333


In [209]:
# # dictionaries that will contain the data to calculate MLE for the specific graph structure   
# childrvdict = {}
# childtotals = {}
# childsettotals = {}
# graphdict = {}
# graphsetdict = {}
# # print(childrvdict,childcategorydict)

# for rv, item in rvdictionary.items():
#     item.clear_count()

# for child, parents in family.items():
#     childcategorydict = {}
#     childsetcategorydict = {}
#     dtlist = []
#     partuplist = []
#     verdictcatlist = []
#     print("child: ",child)
    
#     for (ind,item) in enumerate(x[[*parents]].values):
#         partuplist.append((tuple(parents),tuple(item)))
#         verdictcatlist.append(x[[child]].values[ind][0])
# #         print(partupverdictlist[ind])
#     childtotals[child] = (partuplist, verdictcatlist)
# #     print( childtotals[child])
#     childsettotals[child] = set(partuplist)
#     print(len(set(partuplist)),len(partuplist))
   
#     for category in rvcategorydict[child]:
#         partuplistcat = []
#         print("category: ",category)
#         childcategorydict[category] = {}

#         for (i,item) in enumerate(x[[*parents]].values):
#             if x[[child]].values[i][0] == category:
#                 partuplistcat.append((tuple(parents),tuple(item)))
#         childcategorydict[category] = partuplistcat
#         childsetcategorydict[category] = set(partuplistcat)
#         print(len(set(childcategorydict[category])),len(childcategorydict[category]))
    
#     graphdict[child] = childcategorydict
#     graphsetdict[child] = childcategorydict
    
# # dictionaries to hold the prelim counts
# countdicttotals = {}
# countcategorydict = {}

# # initializing counts to 0
# for child, partuplist in childsettotals.items():
#     for partup in partuplist:
#         countdicttotals[partup] = (0, child)
        
# # sum all occurances of each parent combination for each child   
# # for child, partuplist in childtotals.items():
# #     for (i,partup) in enumerate(partuplist[0]):
# #         countdicttotals[partup] = (countdicttotals[partup][0] + 1,child)
# #         if countdicttotals[partup][0] > 1:
# #             print( partup,partuplist[1][i],countdicttotals[partup][0])
        

        
# for child, childcategorydict in graphdict.items():
#     for category, partuplist in childcategorydict.items():
#         tempcountcategorydict = {}
#         for partup in partuplist:
#             tempcountcategorydict[partup] = (0, child)
#         countcategorydict[category] = tempcountcategorydict

# for child, childcategorydict in graphdict.items():
#     for category, partuplist in childcategorydict.items():
#         tempcountcategorydict = countcategorydict[category]
#         for partup in partuplist:
#             countdicttotals[partup] = (countdicttotals[partup][0] + 1,child)
#             if category == countcategorydict[partup][2]:
#                 countcategorydict[partup] = (tempcountcategorydict[partup][0] + 1, child)
#         countcategorydict[category] = tempcountcategorydict
            

# mledict = {}      
# for child, childcategorydict in graphdict.items():
#     childmledict = {}
#     for category, partuplist in childcategorydict.items():
#         for partup in partuplist:
#             newtup = (tuple([child]+[par for par in partup[0]]),tuple([category]+[cat for cat in partup[1]]))
#             if countdicttotals[partup][0]>2:
#                 print(countcategorydict[category][partup][0],countdicttotals[partup][0],category)
#             childmledict[newtup] = countcategorydict[category][partup][0]/countdicttotals[partup][0]
            
#     mledict[child] = childmledict

# # for entry,condprob in mledict.items():
# #     print(entry,condprob)